# RAG pipeline with Unstructured files

In [1]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
%pip install -q unstructured["all-docs"]==0.12.5

Selecting previously unselected package poppler-utils.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up tess

# Example: PDF file

In [2]:
from unstructured.partition.pdf import partition_pdf

In [9]:
filename = "RAG_best_practices1-2.pdf"
pdf_elements = partition_pdf(filename=filename, strategy="fast")

In [10]:
from collections import Counter

display(Counter(type(element) for element in pdf_elements))

Counter({unstructured.documents.elements.Text: 5,
         unstructured.documents.elements.Title: 11,
         unstructured.documents.elements.NarrativeText: 8,
         unstructured.documents.elements.Footer: 2,
         unstructured.documents.elements.ListItem: 2})

In [44]:
for element in pdf_elements[:]:
    print(f"{element.category.upper()}: {element.text}")

UNCATEGORIZEDTEXT: 4 2 0 2
TITLE: l u J
UNCATEGORIZEDTEXT: 1
TITLE: ] L C . s c [
UNCATEGORIZEDTEXT: 1 v 9 1 2 1 0 . 7 0 4 2 : v i X r a
TITLE: Searching for Best Practices in Retrieval-Augmented Generation
UNCATEGORIZEDTEXT: Xiaohua Wang, Zhenghua Wang, Xuan Gao, Feiran Zhang, Yixin Wu, Zhibo Xu, Tianyuan Shi, Zhengyuan Wang, Shizheng Li, Qi Qian, Ruicheng Yin, Changze Lv, Xiaoqing Zheng∗, Xuanjing Huang School of Computer Science, Fudan University, Shanghai, China Shanghai Key Laboratory of Intelligent Information Processing {xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn {zhengxq,xjhuang}@fudan.edu.cn
TITLE: Abstract
NARRATIVETEXT: Retrieval-augmented generation (RAG) techniques have proven to be effective in integrating up-to-date information, mitigating hallucinations, and enhancing response quality, particularly in specialized domains. While many RAG approaches have been proposed to enhance large language models through query-dependent retrievals, these approaches still suffer from

### Document Layout Detection (DLD) Method

In [12]:
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements

In [13]:
from unstructured_client import UnstructuredClient

client = UnstructuredClient(
    api_key_auth='free-api-key',
    server='free-api'
)


In [14]:
with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
)

try:
    resp = client.general.partition(req)
    dld_elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [19]:
for element in dld_elements[:20]:
    print(f"{element.category.upper()}: {element.text}")

TITLE: Searching for Best Practices in Retrieval-Augmented Generation
HEADER: 4 2 0 2 l u J 1 ] L C . s c [ 1 v 9 1 2 1 0 . 7 0 4 2 : v i X r a
UNCATEGORIZEDTEXT: Xiaohua Wang, Zhenghua Wang, Xuan Gao, Feiran Zhang, Yixin Wu, Zhibo Xu, Tianyuan Shi, Zhengyuan Wang, Shizheng Li, Qi Qian, Ruicheng Yin, Changze Lv, Xiaoqing Zheng∗, Xuanjing Huang School of Computer Science, Fudan University, Shanghai, China Shanghai Key Laboratory of Intelligent Information Processing {xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn {zhengxq,xjhuang}@fudan.edu.cn
TITLE: Abstract
NARRATIVETEXT: Retrieval-augmented generation (RAG) techniques have proven to be effective in integrating up-to-date information, mitigating hallucinations, and enhancing response quality, particularly in specialized domains. While many RAG approaches have been proposed to enhance large language models through query-dependent retrievals, these approaches still suffer from their complex implementation and prolonged response times. Typ

In [16]:
import json

# Salva elementos em um JSON

data = [{"category": element.category, "text": element.text} for element in dld_elements]
with open("elementos.json", "w") as outfile:
    json.dump(data, outfile)

In [17]:
from IPython.display import display

with open("elementos.json", "r") as infile:
    data = json.load(infile)

for element in data:
    display(element)


{'category': 'Title',
 'text': 'Searching for Best Practices in Retrieval-Augmented Generation'}

{'category': 'Header',
 'text': '4 2 0 2 l u J 1 ] L C . s c [ 1 v 9 1 2 1 0 . 7 0 4 2 : v i X r a'}

{'category': 'UncategorizedText',
 'text': 'Xiaohua Wang, Zhenghua Wang, Xuan Gao, Feiran Zhang, Yixin Wu, Zhibo Xu, Tianyuan Shi, Zhengyuan Wang, Shizheng Li, Qi Qian, Ruicheng Yin, Changze Lv, Xiaoqing Zheng∗, Xuanjing Huang School of Computer Science, Fudan University, Shanghai, China Shanghai Key Laboratory of Intelligent Information Processing {xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn {zhengxq,xjhuang}@fudan.edu.cn'}

{'category': 'Title', 'text': 'Abstract'}

{'category': 'NarrativeText',
 'text': 'Retrieval-augmented generation (RAG) techniques have proven to be effective in integrating up-to-date information, mitigating hallucinations, and enhancing response quality, particularly in specialized domains. While many RAG approaches have been proposed to enhance large language models through query-dependent retrievals, these approaches still suffer from their complex implementation and prolonged response times. Typically, a RAG workflow involves multiple processing steps, each of which can be executed in various ways. Here, we investigate existing RAG approaches and their potential combinations to identify optimal RAG practices. Through extensive experiments, we suggest several strategies for deploying RAG that balance both performance and efficiency. Moreover, we demonstrate that multimodal retrieval techniques can significantly enhance question-answering capabilities about visual inputs and accelerate the generation of multimodal content us

...

{'category': 'NarrativeText',
 'text': 'What adds complexity and challenge is the variability in implementing each processing step. For example, in retrieving relevant documents for an input query, various methods can be employed. One approach involves rewriting the query first and using the rewritten queries for retrieval [9]. Alternatively, pseudo-responses to the query can be generated first, and the similarity between these pseudo-responses and the backend documents can be compared for retrieval [10]. Another option is to directly employ embedding models, typically trained in a contrastive manner using positive and negative query-response pairs [11, 12]. The techniques chosen for each step and their combinations significantly impact both the effectiveness and efficiency of RAG systems. To the best of our knowledge, there has been no systematic effort to pursue the optimal implementation of RAG, particularly for the entire RAG workflow.'}

{'category': 'NarrativeText',
 'text': 'In this study, we aim to identify the best practices for RAG through extensive experimentation. Given the infeasibility of testing all possible combinations of these methods, we adopt a three-step approach to identify optimal RAG practices. First, we compare representative methods for each RAG step (or module) and select up to three of the best-performing methods. Next, we evaluate the impact of each method on the overall RAG performance by testing one method at a time for an individual step, while keeping the other RAG modules unchanged. This allows us to determine the most effective method for each step based on its contribution and interaction with other modules during response generation. Once the best method is chosen for a module, it is used in subsequent experiments. Finally, we empirically explore a few promising combinations suitable for different application scenarios where efficiency might be prioritized over performance, or vice versa

{'category': 'NarrativeText',
 'text': 'The contributions of this study are three-fold:'}

{'category': 'ListItem',
 'text': '• Through extensive experimentation, we thoroughly investigated existing RAG approaches and their combinations to identify and recommend optimal RAG practices.'}

{'category': 'Footer', 'text': '2'}

# RAG pipeline with Langchain

In [20]:
! pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.5 MB/s eta 0:0

In [21]:
! pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.4 MB/s eta 0:00:00


In [23]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 13.2 MB/s eta 0:00:00


In [24]:
from langchain_community.document_loaders import JSONLoader

In [30]:
loader = JSONLoader(
    file_path='elementos.json',
    jq_schema='.[]',
    content_key='text')

data = loader.load()

In [32]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [35]:
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://peplink.openai.azure.com/",
    openai_api_version="2024-02-01",
    api_key="api-key",
    azure_deployment="chatExcel"
)

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [41]:
chat = AzureChatOpenAI(
    azure_endpoint="azure_endpoint",
    openai_api_version="2024-02-01",
    openai_api_key="api-key",
    model="GPT4o",
    temperature=0
)

In [45]:
retriever = vectorstore.as_retriever()

template = """
    Você é um assistente especialista, use os seguintes trechos recuperados \
    para responder as perguntas.
    Se não souber a resposta, diga que não sabe.

    Pergunta: {pergunta}
    Contexto: {contexto}
    Resposta:
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
        {"contexto": retriever, "pergunta": RunnablePassthrough()}
        | prompt
        | chat
        | StrOutputParser()
)

In [46]:
query = "De um resumo do artigo Searching for Best Practices in Retrieval-Augmented Generation"
print(rag_chain.invoke(query))

O artigo "Searching for Best Practices in Retrieval-Augmented Generation" investiga as melhores práticas para a geração aumentada por recuperação (RAG). As técnicas de RAG têm se mostrado eficazes na integração de informações atualizadas, mitigação de alucinações e melhoria da qualidade das respostas, especialmente em domínios especializados. No entanto, essas abordagens ainda enfrentam desafios devido à sua implementação complexa e tempos de resposta prolongados.

O estudo examina as contribuições de cada componente do fluxo de trabalho de RAG e fornece insights sobre práticas ideais por meio de experimentação extensiva. Métodos opcionais para cada componente são considerados, com as melhores seleções identificadas empiricamente. Além disso, o artigo sugere várias estratégias para implantar RAG que equilibram desempenho e eficiência. Também é demonstrado que técnicas de recuperação multimodal podem melhorar significativamente as capacidades de resposta a perguntas sobre entradas visua